In [ ]:
# Install required packages
%pip install -r requirements.txt

In [ ]:
#dotenv module
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict


In [ ]:
from langchain.chat_models import init_chat_model


# Gemini
llm = init_chat_model("google_genai:gemini-2.5-pro")

# OpenAI
# llm = init_chat_model("openai:gpt-4.1")

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results = 2)
tools = [tool]

In [ ]:

from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)


llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools= [tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [ ]:
# We can easily display the graph we have created
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"messages": ["What is the weather in San Francisco?"]})
print(result['messages'][-1].content)
print(result)